In [1]:
from jupyterthemes import get_themes
import jupyterthemes as jt
from jupyterthemes.stylefx import set_nb_theme

set_nb_theme('chesterish')

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import imshowpair
import matplotlib.patches as mpatches
from skimage import data
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square, opening
from skimage.color import label2rgb
import statistics
import glob
import pandas as pd

### 660 nm
# Noise images
imdr = '/Volumes/KURAPIKA4/Master_Thesis/Iron dataset/Monochrome/110219/660/Noise/'
files_1 = []
[files_1.extend(glob.glob(imdr + '*.tif'))]
images_1 = [cv.imread(file, 0) for file in files_1]
noise_1 = np.zeros((2048, 2448))
for img in images_1:
    noise_1 += img
noise_1 = noise_1/len(images_1)
# Blank images
imdr = '/Volumes/KURAPIKA4/Master_Thesis/Iron dataset/Monochrome/110219/660/Blank/'
files_1 = []
[files_1.extend(glob.glob(imdr + '*.tif'))]
images_1 = [cv.imread(file, 0) for file in files_1]
blank_1 = np.zeros((2048, 2448))
for img in images_1:
    blank_1 += img
blank_1 = blank_1/len(images_1)
# Read all images
imdr = '/Volumes/KURAPIKA4/Master_Thesis/Iron dataset/Monochrome/110219/660/Samples/'
files = []
[files.extend(glob.glob(imdr + '*.tif'))]
images = [cv.imread(file, 0) for file in files]
# print(len(images))
blood_mean = []
blood_median = []
blood_mode = []
bld_mean_bld_mean = []
bld_median_bld_median = []
bld_mode_bld_mode = []
for img in images:
    # illumination correction
    img_ill = (img-noise_1)/(blank_1-noise_1)
#     plt.subplot(1,2,1), plt.imshow(img, cmap = 'gray')
#     plt.subplot(1,2,2), plt.imshow(img_ill, cmap = 'gray')
#     plt.show()
  
    # apply threshold to get the binary image
    th = threshold_otsu(img)
    thg = img > th
    th1 = np.invert(thg)
    
    img_I = img*th1
    img_BG = img_ill*thg
    # apply threshold
    thresh = threshold_otsu(img_I)
    bw = closing(img_I > thresh, square(1))
    bw = opening(bw, square(2))
    # remove artifacts connected to image border
    cleared = clear_border(bw)
    # label image regions
    label_image = label(cleared)
    for region in regionprops(label_image):
        # take regions with large enough areas
        if region.area >= 10000:
            # find the bounding box
            minr, minc, maxr, maxc = region.bbox
    # crop the images to the found bounding box
    img_fin = (cleared*img_ill*th1)[minr:maxr, minc:maxc]
    img_fig = img_BG[minr:maxr, minc:maxc]
    
    plt.subplot(1,2,1), plt.imshow(img_fin, cmap = 'gray')
    plt.subplot(1,2,2), plt.imshow(img_fig, cmap = 'gray')
    plt.show()

    # Find the mean, median and mode of the blood spot
    blood_mean.append(np.mean(img_fin[img_fin>0]))
    blood_median.append(np.median(img_fin[img_fin>0]))
    blood_mode.append(statistics.mode(img_fin[img_fin>0]))
    bld_mean_bld_mean.append(np.mean(img_fin[img_fin>0])/np.mean(img_fig[img_fig>0]))
    bld_median_bld_median.append(np.median(img_fin[img_fin>0])/np.median(img_fig[img_fig>0]))
    bld_mode_bld_mode.append(statistics.mode(img_fin[img_fin>0])/statistics.mode(img_fig[img_fig>0]))

# print(blood_mean)
# print(blood_median)
# print(blood_mode)
# print(bld_mean_bld_mean)
# print(bld_median_bld_median)
# print(bld_mode_bld_mode)
df_660 = pd.DataFrame(np.column_stack([blood_mean[::2], blood_median[::2], blood_mode[::2], bld_mean_bld_mean[::2], bld_median_bld_median[::2], bld_mode_bld_mode[::2], blood_mean[1::2], blood_median[1::2], blood_mode[1::2], bld_mean_bld_mean[1::2], bld_median_bld_median[1::2], bld_mode_bld_mode[1::2]]), columns = ['Mean of Blood pixels', 'Median of Blood pixels', 'Mode of Blood pixels', 'Division Mean of Blood pixels by BG', 'Division Median of Blood pixels by BG', 'Division Mode of Blood pixels by BG', 'Mean of Blood pixels_2', 'Median of Blood pixels_2', 'Mode of Blood pixels_2', 'Division Mean of Blood pixels by BG_2', 'Division Median of Blood pixels by BG_2', 'Division Mode of Blood pixels by BG_2'])
# print(df_660)

In [ ]:
# import cv2 as cv
# import numpy as np
# import matplotlib.pyplot as plt
# import imshowpair
# import matplotlib.patches as mpatches
# from skimage import data
# from skimage.filters import threshold_otsu
# from skimage.segmentation import clear_border
# from skimage.measure import label, regionprops
# from skimage.morphology import closing, square, opening
# from skimage.color import label2rgb
# import statistics
# import glob
# import pandas as pd

# ### 780 nm
# # Noise images
# imdr = '/Volumes/KURAPIKA4/Master_Thesis/Iron dataset/Monochrome/110219/780/Noise/'
# files_1 = []
# [files_1.extend(glob.glob(imdr + '*.tif'))]
# images_1 = [cv.imread(file, 0) for file in files_1]
# noise_1 = np.zeros((2048, 2448))
# for img in images_1:
#     noise_1 += img
# noise_1 = noise_1/len(images_1)
# # Blank images
# imdr = '/Volumes/KURAPIKA4/Master_Thesis/Iron dataset/Monochrome/110219/780/Blank/'
# files_1 = []
# [files_1.extend(glob.glob(imdr + '*.tif'))]
# images_1 = [cv.imread(file, 0) for file in files_1]
# blank_1 = np.zeros((2048, 2448))
# for img in images_1:
#     blank_1 += img
# blank_1 = blank_1/len(images_1)

# img = cv.imread('/Volumes/KURAPIKA4/Master_Thesis/Iron dataset/Monochrome/110219/780/Samples/S57-780-2.tif', 0)

# # illumination correction
# img_ill = (img-noise_1)/(blank_1-noise_1)
# plt.subplot(1,2,1), plt.imshow(img, cmap = 'gray')
# plt.subplot(1,2,2), plt.imshow(img_ill, cmap = 'gray')
# plt.show()
    
# # apply threshold to get the binary image
# th = threshold_otsu(img)
# thg = img > th
# th1 = np.invert(thg)

# img_I = img*th1
# img_BG = img_ill*thg
# # apply threshold
# thresh = threshold_otsu(img_I)
# bw = closing(img_I > thresh, square(1))
# bw = opening(bw, square(2))

# plt.subplot(1,2,1), plt.imshow(th1, cmap = 'gray')
# plt.subplot(1,2,2), plt.imshow(thg, cmap = 'gray')
# plt.show()

# # label_image = label(cleared)
# # label image regions
# mask = np.ones(bw.shape)
# mask[1400:2047, 0:200] = 0
# # print(mask)
# mask = mask.astype(bool)

# plt.imshow(mask, cmap = 'gray')
# plt.show()

# # remove artifacts connected to image border
# cleared = clear_border(bw, mask = mask)
# label_image = label(cleared)
# # label_image = label(bw)

# plt.subplot(1,2,1), plt.imshow(cleared, cmap = 'gray')
# plt.subplot(1,2,2), plt.imshow(label_image, cmap = 'gray')
# plt.show()

# for region in regionprops(label_image):
#     # take regions with large enough areas
#     if region.area >= 100000:
#         # find the bounding box
#         minr, minc, maxr, maxc = region.bbox
# # crop the images to the found bounding box
# img_fin = (cleared*img_ill*th1)[minr:maxr, minc:maxc]
# img_fig = img_BG[minr:maxr, minc:maxc]
    
# plt.subplot(1,2,1), plt.imshow(img_fin, cmap = 'gray')
# plt.subplot(1,2,2), plt.imshow(img_fig, cmap = 'gray')
# plt.show()

In [ ]:
# import cv2 as cv
# import numpy as np
# import matplotlib.pyplot as plt
# import imshowpair
# import matplotlib.patches as mpatches
# from skimage import data
# from skimage.filters import threshold_otsu
# from skimage.segmentation import clear_border
# from skimage.measure import label, regionprops
# from skimage.morphology import closing, square, opening
# from skimage.color import label2rgb
# import statistics
# import glob
# import pandas as pd

# ### 800 nm
# # Noise images
# imdr = '/Volumes/KURAPIKA4/Master_Thesis/Iron dataset/Monochrome/110219/800/Noise/'
# files_1 = []
# [files_1.extend(glob.glob(imdr + '*.tif'))]
# images_1 = [cv.imread(file, 0) for file in files_1]
# noise_1 = np.zeros((2048, 2448))
# for img in images_1:
#     noise_1 += img
# noise_1 = noise_1/len(images_1)
# # Blank images
# imdr = '/Volumes/KURAPIKA4/Master_Thesis/Iron dataset/Monochrome/110219/800/Blank/'
# files_1 = []
# [files_1.extend(glob.glob(imdr + '*.tif'))]
# images_1 = [cv.imread(file, 0) for file in files_1]
# blank_1 = np.zeros((2048, 2448))
# for img in images_1:
#     blank_1 += img
# blank_1 = blank_1/len(images_1)

# img = cv.imread('/Volumes/KURAPIKA4/110219/800/Samples/S09-800-1.tif', 0)

# # illumination correction
# img_ill = (img-noise_1)/(blank_1-noise_1)
# plt.subplot(1,2,1), plt.imshow(img, cmap = 'gray')
# plt.subplot(1,2,2), plt.imshow(img_ill, cmap = 'gray')
# plt.show()
    
# # apply threshold to get the binary image
# th = threshold_otsu(img)
# thg = img > th
# th1 = np.invert(thg)

# img_I = img*th1
# img_BG = img_ill*thg
# # apply threshold
# thresh = threshold_otsu(img_I)
# bw = closing(img_I > thresh, square(1))
# bw = opening(bw, square(2))

# plt.subplot(1,2,1), plt.imshow(th1, cmap = 'gray')
# plt.subplot(1,2,2), plt.imshow(thg, cmap = 'gray')
# plt.show()

# # label_image = label(cleared)
# # label image regions
# mask = np.ones(bw.shape)
# mask[1400:2047, 0:200] = 0
# # print(mask)
# mask = mask.astype(bool)

# plt.imshow(mask, cmap = 'gray')
# plt.show()

# # remove artifacts connected to image border
# cleared = clear_border(bw, mask = mask)
# label_image = label(cleared)
# # label_image = label(bw)

# plt.subplot(1,2,1), plt.imshow(cleared, cmap = 'gray')
# plt.subplot(1,2,2), plt.imshow(label_image, cmap = 'gray')
# plt.show()

# for region in regionprops(label_image):
#     # take regions with large enough areas
#     if region.area >= 100000:
#         # find the bounding box
#         minr, minc, maxr, maxc = region.bbox
# # crop the images to the found bounding box
# img_fin = (cleared*img_ill*th1)[minr:maxr, minc:maxc]
# img_fig = img_BG[minr:maxr, minc:maxc]
    
# plt.subplot(1,2,1), plt.imshow(img_fin, cmap = 'gray')
# plt.subplot(1,2,2), plt.imshow(img_fig, cmap = 'gray')
# plt.show()

In [ ]:
# import cv2 as cv
# import numpy as np
# import matplotlib.pyplot as plt
# import imshowpair
# import matplotlib.patches as mpatches
# from skimage import data
# from skimage.filters import threshold_otsu
# from skimage.segmentation import clear_border
# from skimage.measure import label, regionprops
# from skimage.morphology import closing, square, opening
# from skimage.color import label2rgb
# import statistics
# import glob
# import pandas as pd

# ### 940 nm
# # Noise images
# imdr = '/Volumes/KURAPIKA4/110219/940/Noise/'
# files_1 = []
# [files_1.extend(glob.glob(imdr + '*.tif'))]
# images_1 = [cv.imread(file, 0) for file in files_1]
# noise_1 = np.zeros((2048, 2448))
# for img in images_1:
#     noise_1 += img
# noise_1 = noise_1/len(images_1)
# # Blank images
# imdr = '/Volumes/KURAPIKA4/110219/940/Blank/'
# files_1 = []
# [files_1.extend(glob.glob(imdr + '*.tif'))]
# images_1 = [cv.imread(file, 0) for file in files_1]
# blank_1 = np.zeros((2048, 2448))
# for img in images_1:
#     blank_1 += img
# blank_1 = blank_1/len(images_1)

# img = cv.imread('/Volumes/KURAPIKA4/110219/940/Samples/S57-940-1.tif', 0)

# # illumination correction
# img_ill = (img-noise_1)/(blank_1-noise_1)
# plt.subplot(1,2,1), plt.imshow(img, cmap = 'gray')
# plt.subplot(1,2,2), plt.imshow(img_ill, cmap = 'gray')
# plt.show()
    
# # apply threshold to get the binary image
# th = threshold_otsu(img)
# thg = img > th
# th1 = np.invert(thg)

# img_I = img*th1
# img_BG = img_ill*thg
# # apply threshold
# thresh = threshold_otsu(img_I)
# bw = closing(img_I > thresh, square(1))
# bw = opening(bw, square(2))

# plt.subplot(1,2,1), plt.imshow(th1, cmap = 'gray')
# plt.subplot(1,2,2), plt.imshow(thg, cmap = 'gray')
# plt.show()

# # label_image = label(cleared)
# # label image regions
# mask = np.ones(bw.shape)
# mask[1400:2047, 0:200] = 0
# # print(mask)
# mask = mask.astype(bool)

# plt.imshow(mask, cmap = 'gray')
# plt.show()

# # remove artifacts connected to image border
# cleared = clear_border(bw, mask = mask)
# label_image = label(cleared)
# # label_image = label(bw)

# plt.subplot(1,2,1), plt.imshow(cleared, cmap = 'gray')
# plt.subplot(1,2,2), plt.imshow(label_image, cmap = 'gray')
# plt.show()

# for region in regionprops(label_image):
#     # take regions with large enough areas
#     if region.area >= 1000000:
#         # find the bounding box
#         minr, minc, maxr, maxc = region.bbox
# # crop the images to the found bounding box
# img_fin = (cleared*img_ill*th1)[minr:maxr, minc:maxc]
# img_fig = img_BG[minr:maxr, minc:maxc]
    
# plt.subplot(1,2,1), plt.imshow(img_fin, cmap = 'gray')
# plt.subplot(1,2,2), plt.imshow(img_fig, cmap = 'gray')
# plt.show()

In [ ]:
### 660 nm by Set
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import imshowpair
import matplotlib.patches as mpatches
from skimage import data
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square, opening
from skimage.color import label2rgb
import statistics
import glob
import pandas as pd
# Read Noise images
imdr = '/Volumes/KURAPIKA4/Master_Thesis/Iron dataset/Monochrome/110219/660/Noise/'
files_1 = []
[files_1.extend(glob.glob(imdr + '*1.tif'))]
images_1 = [cv.imread(file, 0) for file in files_1]
noise_1 = np.zeros((2048, 2448))
for img in images_1:
    noise_1 += img
noise_1 = noise_1/len(images_1)
files_2 = []
[files_2.extend(glob.glob(imdr + '*2.tif'))]
images_2 = [cv.imread(file, 0) for file in files_2]
noise_2 = np.zeros((2048, 2448))
for img in images_2:
    noise_2 += img
noise_2 = noise_2/len(images_2)
# Read Blank images
imdr = '/Volumes/KURAPIKA4/Master_Thesis/Iron dataset/Monochrome/110219/660/Blank/'
files_1 = []
[files_1.extend(glob.glob(imdr + '*1.tif'))]
images_1 = [cv.imread(file, 0) for file in files_1]
blank_1 = np.zeros((2048, 2448))
for img in images_1:
    blank_1 += img
blank_1 = blank_1/len(images_1)
files_2 = []
[files_2.extend(glob.glob(imdr + '*2.tif'))]
images_2 = [cv.imread(file, 0) for file in files_2]
blank_2 = np.zeros((2048, 2448))
for img in images_2:
    blank_2 += img
blank_2 = blank_2/len(images_2)
# print(np.bincount(np.ravel(noise_1 == noise_2)))
# print(np.bincount(np.ravel(blank_1 == blank_2)))
### Set 1
# Read all images
imdr = '/Volumes/KURAPIKA4/Master_Thesis/Iron dataset/Monochrome/110219/660/Samples/'
files = []
[files.extend(glob.glob(imdr + '*1.tif'))]
images = [cv.imread(file, 0) for file in files]
blood_mean = []
blood_median = []
blood_mode = []
bld_mean_bld_mean = []
bld_median_bld_median = []
bld_mode_bld_mode = []
for img in images:
    # illumination correction
    img_ill = (img-noise_1)/(blank_1-noise_1)
    
    # apply threshold to get the binary image
    th = threshold_otsu(img)
    thg = img > th
    th1 = np.invert(thg)
    
    img_I = img*th1
    img_BG = img_ill*thg
    # apply threshold
    thresh = threshold_otsu(img_I)
    bw = closing(img_I > thresh, square(1))
    bw = opening(bw, square(2))
    # remove artifacts connected to image border
    cleared = clear_border(bw)
    # label image regions
    label_image = label(cleared)
    for region in regionprops(label_image):
        # take regions with large enough areas
        if region.area >= 10000:
            # find the bounding box
            minr, minc, maxr, maxc = region.bbox
    # crop the images to the found bounding box
    img_fin = (cleared*img_ill*th1)[minr:maxr, minc:maxc]
    img_fig = img_BG[minr:maxr, minc:maxc]
    
    plt.subplot(1,2,1), plt.imshow(img_fin, cmap = 'gray')
    plt.subplot(1,2,2), plt.imshow(img_fig, cmap = 'gray')
    plt.show()

    # Find the mean, median and mode of the blood spot
    blood_mean.append(np.mean(img_fin[img_fin>0]))
    blood_median.append(np.median(img_fin[img_fin>0]))
    blood_mode.append(statistics.mode(img_fin[img_fin>0]))
    bld_mean_bld_mean.append(np.mean(img_fin[img_fin>0])/np.mean(img_fig[img_fig>0]))
    bld_median_bld_median.append(np.median(img_fin[img_fin>0])/np.median(img_fig[img_fig>0]))
    bld_mode_bld_mode.append(statistics.mode(img_fin[img_fin>0])/statistics.mode(img_fig[img_fig>0]))
df_660_1 = pd.DataFrame(np.column_stack([blood_mean, blood_median, blood_mode, bld_mean_bld_mean, bld_median_bld_median, bld_mode_bld_mode]), columns = ['Mean of Blood pixels', 'Median of Blood pixels', 'Mode of Blood pixels', 'Division Mean of Blood pixels by BG', 'Division Median of Blood pixels by BG', 'Division Mode of Blood pixels by BG'])
## Set 2
files = []
[files.extend(glob.glob(imdr + '*2.tif'))]
images = [cv.imread(file, 0) for file in files]
blood_mean = []
blood_median = []
blood_mode = []
bld_mean_bld_mean = []
bld_median_bld_median = []
bld_mode_bld_mode = []
for img in images:
    # illumination correction
    img_ill = (img-noise_2)/(blank_2-noise_2)
    
    # apply threshold to get the binary image
    th = threshold_otsu(img)
    thg = img > th
    th1 = np.invert(thg)
    
    img_I = img*th1
    img_BG = img_ill*thg
    # apply threshold
    thresh = threshold_otsu(img_I)
    bw = closing(img_I > thresh, square(1))
    bw = opening(bw, square(2))
    # remove artifacts connected to image border
    cleared = clear_border(bw)
    # label image regions
    label_image = label(cleared)
    for region in regionprops(label_image):
        # take regions with large enough areas
        if region.area >= 10000:
            # find the bounding box
            minr, minc, maxr, maxc = region.bbox
    # crop the images to the found bounding box
    img_fin = (cleared*img_ill*th1)[minr:maxr, minc:maxc]
    img_fig = img_BG[minr:maxr, minc:maxc]
    
    plt.subplot(1,2,1), plt.imshow(img_fin, cmap = 'gray')
    plt.subplot(1,2,2), plt.imshow(img_fig, cmap = 'gray')
    plt.show()

    # Find the mean, median and mode of the blood spot
    blood_mean.append(np.mean(img_fin[img_fin>0]))
    blood_median.append(np.median(img_fin[img_fin>0]))
    blood_mode.append(statistics.mode(img_fin[img_fin>0]))
    bld_mean_bld_mean.append(np.mean(img_fin[img_fin>0])/np.mean(img_fig[img_fig>0]))
    bld_median_bld_median.append(np.median(img_fin[img_fin>0])/np.median(img_fig[img_fig>0]))
    bld_mode_bld_mode.append(statistics.mode(img_fin[img_fin>0])/statistics.mode(img_fig[img_fig>0]))
df_660_2 = pd.DataFrame(np.column_stack([blood_mean, blood_median, blood_mode, bld_mean_bld_mean, bld_median_bld_median, bld_mode_bld_mode]), columns = ['Mean of Blood pixels', 'Median of Blood pixels', 'Mode of Blood pixels', 'Division Mean of Blood pixels by BG', 'Division Median of Blood pixels by BG', 'Division Mode of Blood pixels by BG'])

In [ ]:
import pickle
with open('DF_660.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([df_660, df_660_1, df_660_2], f)

In [ ]:
### Linear Regression
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import scipy.stats as stat
import pickle
# import scipy.io
# Hb = scipy.io.loadmat('Hb_values.mat')
# Hb_ = scipy.io.loadmat('hem_val.mat')
# Hb__ = scipy.io.loadmat('HB.mat')
# print(Hb)
# print(Hb_)
# print(Hb__)
# Getting back the objects:
with open('DF_660.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    df_660, df_660_1, df_660_2 = pickle.load(f)
data = pd.read_csv('HB.csv')
# print(data.head())
# print(data['Average SLS&HemoCue'])
Hb = data['Average SLS&HemoCue']
cv = data['CV_']
HB = pd.concat([Hb, cv], axis = 1)
print(np.corrcoef(df_660['Mean of Blood pixels'], Hb))
print(np.corrcoef(df_660['Median of Blood pixels'], Hb))
print(np.corrcoef(df_660['Mode of Blood pixels'], Hb))
print(np.corrcoef(df_660['Division Mean of Blood pixels by BG'], Hb))
print(np.corrcoef(df_660['Division Median of Blood pixels by BG'], Hb))
print(np.corrcoef(df_660['Division Mode of Blood pixels by BG'], Hb))
print(np.corrcoef(df_660['Mean of Blood pixels'], Hb))
print(np.corrcoef(df_660['Median of Blood pixels'], Hb))
print(np.corrcoef(df_660['Mode of Blood pixels'], Hb))
print(np.corrcoef(df_660['Division Mean of Blood pixels by BG'], Hb))
print(np.corrcoef(df_660['Division Median of Blood pixels by BG'], Hb))
print(np.corrcoef(df_660['Division Mode of Blood pixels by BG'], Hb))
print(stat.spearmanr(df_660['Mean of Blood pixels'], Hb))
print(stat.spearmanr(df_660['Median of Blood pixels'], Hb))
print(stat.spearmanr(df_660['Mode of Blood pixels'], Hb))
print(stat.spearmanr(df_660['Division Mean of Blood pixels by BG'], Hb))
print(stat.spearmanr(df_660['Division Median of Blood pixels by BG'], Hb))
print(stat.spearmanr(df_660['Division Mode of Blood pixels by BG'], Hb))

print(np.corrcoef(df_660['Mean of Blood pixels_2'], Hb))
print(np.corrcoef(df_660['Median of Blood pixels_2'], Hb))
print(np.corrcoef(df_660['Mode of Blood pixels_2'], Hb))
print(np.corrcoef(df_660['Division Mean of Blood pixels by BG_2'], Hb))
print(np.corrcoef(df_660['Division Median of Blood pixels by BG_2'], Hb))
print(np.corrcoef(df_660['Division Mode of Blood pixels by BG_2'], Hb))
print(stat.spearmanr(df_660['Mean of Blood pixels_2'], Hb))
print(stat.spearmanr(df_660['Median of Blood pixels_2'], Hb))
print(stat.spearmanr(df_660['Mode of Blood pixels_2'], Hb))
print(stat.spearmanr(df_660['Division Mean of Blood pixels by BG_2'], Hb))
print(stat.spearmanr(df_660['Division Median of Blood pixels by BG_2'], Hb))
print(stat.spearmanr(df_660['Division Mode of Blood pixels by BG_2'], Hb))
reg = LinearRegression().fit(df_660, Hb)
print(reg.coef_)
print(reg.intercept_)
# reg.predict
# print(df_660.head())
print(np.corrcoef(df_660_1['Mean of Blood pixels'], Hb))
print(np.corrcoef(df_660_1['Median of Blood pixels'], Hb))
print(np.corrcoef(df_660_1['Mode of Blood pixels'], Hb))
print(np.corrcoef(df_660_1['Division Mean of Blood pixels by BG'], Hb))
print(np.corrcoef(df_660_1['Division Median of Blood pixels by BG'], Hb))
print(np.corrcoef(df_660_1['Division Mode of Blood pixels by BG'], Hb))
print(stat.spearmanr(df_660_1['Mean of Blood pixels'], Hb))
print(stat.spearmanr(df_660_1['Median of Blood pixels'], Hb))
print(stat.spearmanr(df_660_1['Mode of Blood pixels'], Hb))
print(stat.spearmanr(df_660_1['Division Mean of Blood pixels by BG'], Hb))
print(stat.spearmanr(df_660_1['Division Median of Blood pixels by BG'], Hb))
print(stat.spearmanr(df_660_1['Division Mode of Blood pixels by BG'], Hb))

print(np.corrcoef(df_660_2['Mean of Blood pixels'], Hb))
print(np.corrcoef(df_660_2['Median of Blood pixels'], Hb))
print(np.corrcoef(df_660_2['Mode of Blood pixels'], Hb))
print(np.corrcoef(df_660_2['Division Mean of Blood pixels by BG'], Hb))
print(np.corrcoef(df_660_2['Division Median of Blood pixels by BG'], Hb))
print(np.corrcoef(df_660_2['Division Mode of Blood pixels by BG'], Hb))
print(stat.spearmanr(df_660_2['Mean of Blood pixels'], Hb))
print(stat.spearmanr(df_660_2['Median of Blood pixels'], Hb))
print(stat.spearmanr(df_660_2['Mode of Blood pixels'], Hb))
print(stat.spearmanr(df_660_2['Division Mean of Blood pixels by BG'], Hb))
print(stat.spearmanr(df_660_2['Division Median of Blood pixels by BG'], Hb))
print(stat.spearmanr(df_660_2['Division Mode of Blood pixels by BG'], Hb))

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
plt.scatter(df_660['Division Mean of Blood pixels by BG'], Hb, s = 150*cv, alpha = 0.6, edgecolors = 'k', c = 'r')
xp = np.linspace(0.5, 0.9, 20)
model = np.poly1d(np.polyfit(df_660['Division Mean of Blood pixels by BG'], Hb, 1))
plt.plot(xp, model(xp), 'k')
i = 1
for a,b in zip(df_660['Division Mean of Blood pixels by BG'], Hb): 
    plt.text(a, b, str(i))
    i += 1
plt.title('Mean of Blood Spot against Hemoglobin Levels', fontsize = 20)
plt.legend(('Linear regression', 'Blood spot'), fontsize = 14)
plt.xlabel('Division of Mean Blood Spot with BG', fontsize = 16)
plt.ylabel('Hemoglobin Levels(g/dL)', fontsize = 16)
plt.show()


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(df_660['Division Mean of Blood pixels by BG'], HB, test_size=0.2, random_state=44)
# print(y_train)
model_1 = np.poly1d(np.polyfit(X_train, y_train['Average SLS&HemoCue'], 1))
model_2 = np.poly1d(np.polyfit(X_train, y_train['Average SLS&HemoCue'], 2))
xp = np.linspace(0.5, 0.9, 20)

print('MAE: ' + str(metrics.mean_absolute_error(y_test['Average SLS&HemoCue'], model_1(X_test))))
print('MSE: ' + str(metrics.mean_squared_error(y_test['Average SLS&HemoCue'], model_1(X_test))))
print('RMSE: ' + str(np.sqrt(metrics.mean_absolute_error(y_test['Average SLS&HemoCue'], model_1(X_test)))))
print('R2_sklearn: ' + str(metrics.r2_score(y_test['Average SLS&HemoCue'], model_1(X_test))))
corr_matrix = np.corrcoef(y_test['Average SLS&HemoCue'], model_1(X_test))
corr = corr_matrix[0, 1]
print('R2_numpy: ' + str(corr**2))
plt.figure(figsize=(12, 8))
plt.scatter(X_test, y_test['Average SLS&HemoCue'], s = 200*y_test['CV_'], alpha = 0.6, edgecolors = 'k', c = 'b' )
plt.plot(xp, model_1(xp), 'r--')
# plt.plot(xp, model_1(xp), 'b-', xp, model_2(xp), 'g--')
i = 0
idx = y_test.index
for a,b in zip(X_test, y_test['Average SLS&HemoCue']): 
    plt.text(a, b, idx[i]+1)
    i += 1
plt.title('Mean of Blood Spot against Hemoglobin Levels', fontsize = 20)
plt.legend(('Linear regression', 'Blood spot'), fontsize = 14)
# plt.legend(('Linear regression', 'Poly2', 'Blood spot'), fontsize = 14)
plt.xlabel('Division of Mean Blood Spot with BG', fontsize = 16)
plt.ylabel('Hemoglobin Levels(g/dL)', fontsize = 16)
plt.grid(color = '0.9', linewidth = 0.5)
plt.show()

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn import metrics, linear_model
from sklearn.model_selection import KFold
import numpy as np

# print(df_660['Division Mean of Blood pixels by BG'])
# print(HB)
# model_ = linear_model.LinearRegression()
kf = KFold(n_splits=5)
# kf.get_n_splits(df_660['Division Mean of Blood pixels by BG'])
i = 1
for train_index, test_index in kf.split(df_660['Division Mean of Blood pixels by BG']):
    X_train, X_test = df_660['Division Mean of Blood pixels by BG'][train_index], df_660['Division Mean of Blood pixels by BG'][test_index]
    y_train, y_test = Hb[train_index], Hb[test_index]
#     cv_train, cv_test = cv[train_index], cv[test_index]

    model_1 = np.poly1d(np.polyfit(X_train, y_train, 1))
    print('R2_sklearn: ' + str(metrics.r2_score(y_test, model_1(X_test))))
    print('RMSE: ' + str(np.sqrt(metrics.mean_absolute_error(y_test, model_1(X_test)))))
    
# model_1 = np.poly1d(np.polyfit(X_train, y_train, 1))
# model_2 = np.poly1d(np.polyfit(X_train, y_train, 2))
# xp = np.linspace(0.5, 0.9, 20)

# for i, score in enumerate(cross_validate(model_1, df_660['Division Mean of Blood pixels by BG'],Hb, cv=5)["test_score"]):
#     print(f"Accuracy for the fold no. {i} on the test set: {score}")

# print('MAE: ' + str(metrics.mean_absolute_error(y_test, model_1(X_test))))
# print('MSE: ' + str(metrics.mean_squared_error(y_test, model_1(X_test))))
# print('RMSE: ' + str(np.sqrt(metrics.mean_absolute_error(y_test, model_1(X_test)))))
# print('R2_sklearn: ' + str(metrics.r2_score(y_test, model_1(X_test))))
# corr_matrix = np.corrcoef(y_test, model_1(X_test))
# corr = corr_matrix[0, 1]
# print('R2_numpy: ' + str(corr**2))

# plt.figure(figsize=(12, 8))
# plt.scatter(X_test, y_test, s = 200*cv_test, alpha = 0.6, edgecolors = 'k', c = 'b' )
# plt.plot(xp, model_1(xp), 'r--')
# # plt.plot(xp, model_1(xp), 'b-', xp, model_2(xp), 'g--')
# i = 0
# idx = y_test.index
# for a,b in zip(X_test, y_test): 
#     plt.text(a, b, idx[i]+1)
#     i += 1
# plt.title('Mean of Blood Spot against Hemoglobin Levels', fontsize = 20)
# plt.legend(('Linear regression', 'Blood spot'), fontsize = 14)
# # plt.legend(('Linear regression', 'Poly2', 'Blood spot'), fontsize = 14)
# plt.xlabel('Division of Mean Blood Spot with BG', fontsize = 16)
# plt.ylabel('Hemoglobin Levels(g/dL)', fontsize = 16)
# plt.grid(color = '0.9', linewidth = 0.5)
# plt.show()
# print(model_1(X_test))
# predicted = model_1(X_test)
# print(type(predicted))

In [ ]:
### 780 nm
# Noise images
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import imshowpair
import matplotlib.patches as mpatches
from skimage import data
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square, opening
from skimage.color import label2rgb
import statistics
import glob
import pandas as pd

imdr = '/Volumes/KURAPIKA4/Master_Thesis/Iron dataset/Monochrome/110219/780/Noise/'
files_1 = []
[files_1.extend(glob.glob(imdr + '*.tif'))]
images_1 = [cv.imread(file, 0) for file in files_1]
noise_1 = np.zeros((2048, 2448))
for img in images_1:
    noise_1 += img
noise_1 = noise_1/len(images_1)
# Blank images
imdr = '/Volumes/KURAPIKA4/Master_Thesis/Iron dataset/Monochrome/110219/780/Blank/'
files_1 = []
[files_1.extend(glob.glob(imdr + '*.tif'))]
images_1 = [cv.imread(file, 0) for file in files_1]
blank_1 = np.zeros((2048, 2448))
for img in images_1:
    blank_1 += img
blank_1 = blank_1/len(images_1)
# Read all images
imdr = '/Volumes/KURAPIKA4/Master_Thesis/Iron dataset/Monochrome/110219/780/Samples/'
files = []
[files.extend(glob.glob(imdr + '*.tif'))]
images = [cv.imread(file, 0) for file in files]
# print(len(images))
blood_mean = []
blood_median = []
blood_mode = []
bld_mean_bld_mean = []
bld_median_bld_median = []
bld_mode_bld_mode = []
for img in images:
    # illumination correction
    img_ill = (img-noise_1)/(blank_1-noise_1)
    
    # apply threshold to get the binary image
    th = threshold_otsu(img)
    thg = img > th
    th1 = np.invert(thg)
    
    img_I = img*th1
    img_BG = img_ill*thg
    # apply threshold
    thresh = threshold_otsu(img_I)
    bw = closing(img_I > thresh, square(1))
    bw = opening(bw, square(2))
    
    mask = np.ones(bw.shape)
    mask[1400:2047, 0:200] = 0
    # print(mask)
    mask = mask.astype(bool)

#     plt.imshow(mask, cmap = 'gray')
#     plt.show()

    # remove artifacts connected to image border
    cleared = clear_border(bw, mask = mask)

    # label image regions
    label_image = label(cleared)
    
#     plt.subplot(1,2,1), plt.imshow(cleared, cmap = 'gray')
#     plt.subplot(1,2,2), plt.imshow(label_image, cmap = 'gray')
#     plt.show()

    for region in regionprops(label_image):
        # take regions with large enough areas
        if region.area >= 100000:
            # find the bounding box
            minr, minc, maxr, maxc = region.bbox
    # crop the images to the found bounding box
    img_fin = (cleared*img_ill*th1)[minr:maxr, minc:maxc]
    img_fig = img_BG[minr:maxr, minc:maxc]

    plt.subplot(1,2,1), plt.imshow(img_fin, cmap = 'gray')
    plt.subplot(1,2,2), plt.imshow(img_fig, cmap = 'gray')
    plt.show()

    # Find the mean, median and mode of the blood spot
    blood_mean.append(np.mean(img_fin[img_fin>0]))
    blood_median.append(np.median(img_fin[img_fin>0]))
    blood_mode.append(statistics.mode(img_fin[img_fin>0]))
    bld_mean_bld_mean.append(np.mean(img_fin[img_fin>0])/np.mean(img_fig[img_fig>0]))
    bld_median_bld_median.append(np.median(img_fin[img_fin>0])/np.median(img_fig[img_fig>0]))
    bld_mode_bld_mode.append(statistics.mode(img_fin[img_fin>0])/statistics.mode(img_fig[img_fig>0]))

# print(blood_mean)
# print(blood_median)
# print(blood_mode)
# print(bld_mean_bld_mean)
# print(bld_median_bld_median)
# print(bld_mode_bld_mode)
df_780 = pd.DataFrame(np.column_stack([blood_mean[::2], blood_median[::2], blood_mode[::2], bld_mean_bld_mean[::2], bld_median_bld_median[::2], bld_mode_bld_mode[::2], blood_mean[1::2], blood_median[1::2], blood_mode[1::2], bld_mean_bld_mean[1::2], bld_median_bld_median[1::2], bld_mode_bld_mode[1::2]]), columns = ['Mean of Blood pixels', 'Median of Blood pixels', 'Mode of Blood pixels', 'Division Mean of Blood pixels by BG', 'Division Median of Blood pixels by BG', 'Division Mode of Blood pixels by BG', 'Mean of Blood pixels_2', 'Median of Blood pixels_2', 'Mode of Blood pixels_2', 'Division Mean of Blood pixels by BG_2', 'Division Median of Blood pixels by BG_2', 'Division Mode of Blood pixels by BG_2'])
# print(df_660)

In [ ]:
### 780 nm by Set
# Read Noise images
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import imshowpair
import matplotlib.patches as mpatches
from skimage import data
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square, opening
from skimage.color import label2rgb
import statistics
import glob
import pandas as pd

imdr = '/Volumes/KURAPIKA4/Master_Thesis/Iron dataset/Monochrome/110219/780/Noise/'
files_1 = []
[files_1.extend(glob.glob(imdr + '*1.tif'))]
images_1 = [cv.imread(file, 0) for file in files_1]
noise_1 = np.zeros((2048, 2448))
for img in images_1:
    noise_1 += img
noise_1 = noise_1/len(images_1)
files_2 = []
[files_2.extend(glob.glob(imdr + '*2.tif'))]
images_2 = [cv.imread(file, 0) for file in files_2]
noise_2 = np.zeros((2048, 2448))
for img in images_2:
    noise_2 += img
noise_2 = noise_2/len(images_2)
# Read Blank images
imdr = '/Volumes/KURAPIKA4/Master_Thesis/Iron dataset/Monochrome/110219/780/Blank/'
files_1 = []
[files_1.extend(glob.glob(imdr + '*1.tif'))]
images_1 = [cv.imread(file, 0) for file in files_1]
blank_1 = np.zeros((2048, 2448))
for img in images_1:
    blank_1 += img
blank_1 = blank_1/len(images_1)
files_2 = []
[files_2.extend(glob.glob(imdr + '*2.tif'))]
images_2 = [cv.imread(file, 0) for file in files_2]
blank_2 = np.zeros((2048, 2448))
for img in images_2:
    blank_2 += img
blank_2 = blank_2/len(images_2)
# print(np.bincount(np.ravel(noise_1 == noise_2)))
# print(np.bincount(np.ravel(blank_1 == blank_2)))
### Set 1
# Read all images
imdr = '/Volumes/KURAPIKA4/Master_Thesis/Iron dataset/Monochrome/110219/780/Samples/'
files = []
[files.extend(glob.glob(imdr + '*1.tif'))]
images = [cv.imread(file, 0) for file in files]
blood_mean = []
blood_median = []
blood_mode = []
bld_mean_bld_mean = []
bld_median_bld_median = []
bld_mode_bld_mode = []
for img in images:
    # illumination correction
    img_ill = (img-noise_1)/(blank_1-noise_1)
    
    # apply threshold to get the binary image
    th = threshold_otsu(img)
    thg = img > th
    th1 = np.invert(thg)
    
    img_I = img*th1
    img_BG = img_ill*thg
    # apply threshold
    thresh = threshold_otsu(img_I)
    bw = closing(img_I > thresh, square(1))
    bw = opening(bw, square(2))
    mask = np.ones(bw.shape)
    mask[1400:2047, 0:200] = 0
    # print(mask)
    mask = mask.astype(bool)
    # remove artifacts connected to image border
    cleared = clear_border(bw, mask = mask)
    # label image regions
    label_image = label(cleared)
    for region in regionprops(label_image):
        # take regions with large enough areas
        if region.area >= 100000:
            # find the bounding box
            minr, minc, maxr, maxc = region.bbox
    # crop the images to the found bounding box
    img_fin = (cleared*img_ill*th1)[minr:maxr, minc:maxc]
    img_fig = img_BG[minr:maxr, minc:maxc]
    
    plt.subplot(1,2,1), plt.imshow(img_fin, cmap = 'gray')
    plt.subplot(1,2,2), plt.imshow(img_fig, cmap = 'gray')
    plt.show()

    # Find the mean, median and mode of the blood spot
    blood_mean.append(np.mean(img_fin[img_fin>0]))
    blood_median.append(np.median(img_fin[img_fin>0]))
    blood_mode.append(statistics.mode(img_fin[img_fin>0]))
    bld_mean_bld_mean.append(np.mean(img_fin[img_fin>0])/np.mean(img_fig[img_fig>0]))
    bld_median_bld_median.append(np.median(img_fin[img_fin>0])/np.median(img_fig[img_fig>0]))
    bld_mode_bld_mode.append(statistics.mode(img_fin[img_fin>0])/statistics.mode(img_fig[img_fig>0]))
df_780_1 = pd.DataFrame(np.column_stack([blood_mean, blood_median, blood_mode, bld_mean_bld_mean, bld_median_bld_median, bld_mode_bld_mode]), columns = ['Mean of Blood pixels', 'Median of Blood pixels', 'Mode of Blood pixels', 'Division Mean of Blood pixels by BG', 'Division Median of Blood pixels by BG', 'Division Mode of Blood pixels by BG'])
## Set 2
files = []
[files.extend(glob.glob(imdr + '*2.tif'))]
images = [cv.imread(file, 0) for file in files]
blood_mean = []
blood_median = []
blood_mode = []
bld_mean_bld_mean = []
bld_median_bld_median = []
bld_mode_bld_mode = []
for img in images:
    # illumination correction
    img_ill = (img-noise_2)/(blank_2-noise_2)
    
    # apply threshold to get the binary image
    th = threshold_otsu(img)
    thg = img > th
    th1 = np.invert(thg)
    
    img_I = img*th1
    img_BG = img_ill*thg
    # apply threshold
    thresh = threshold_otsu(img_I)
    bw = closing(img_I > thresh, square(1))
    bw = opening(bw, square(2))
    mask = np.ones(bw.shape)
    mask[1400:2047, 0:200] = 0
    # print(mask)
    mask = mask.astype(bool)
    # remove artifacts connected to image border
    cleared = clear_border(bw, mask = mask)
    # label image regions
    label_image = label(cleared)
    for region in regionprops(label_image):
        # take regions with large enough areas
        if region.area >= 100000:
            # find the bounding box
            minr, minc, maxr, maxc = region.bbox
    # crop the images to the found bounding box
    img_fin = (cleared*img_ill*th1)[minr:maxr, minc:maxc]
    img_fig = img_BG[minr:maxr, minc:maxc]
    
    plt.subplot(1,2,1), plt.imshow(img_fin, cmap = 'gray')
    plt.subplot(1,2,2), plt.imshow(img_fig, cmap = 'gray')
    plt.show()

    # Find the mean, median and mode of the blood spot
    blood_mean.append(np.mean(img_fin[img_fin>0]))
    blood_median.append(np.median(img_fin[img_fin>0]))
    blood_mode.append(statistics.mode(img_fin[img_fin>0]))
    bld_mean_bld_mean.append(np.mean(img_fin[img_fin>0])/np.mean(img_fig[img_fig>0]))
    bld_median_bld_median.append(np.median(img_fin[img_fin>0])/np.median(img_fig[img_fig>0]))
    bld_mode_bld_mode.append(statistics.mode(img_fin[img_fin>0])/statistics.mode(img_fig[img_fig>0]))
df_780_2 = pd.DataFrame(np.column_stack([blood_mean, blood_median, blood_mode, bld_mean_bld_mean, bld_median_bld_median, bld_mode_bld_mode]), columns = ['Mean of Blood pixels', 'Median of Blood pixels', 'Mode of Blood pixels', 'Division Mean of Blood pixels by BG', 'Division Median of Blood pixels by BG', 'Division Mode of Blood pixels by BG'])

In [ ]:
with open('DF_780.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([df_780, df_780_1, df_780_2], f)

In [ ]:
### Linear Regression
with open('DF_780.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    df_780, df_780_1, df_780_2 = pickle.load(f)
    
df = df_780
df_1 = df_780_1
df_2 = df_780_2
print(np.corrcoef(df['Mean of Blood pixels'], Hb))
print(np.corrcoef(df['Median of Blood pixels'], Hb))
print(np.corrcoef(df['Mode of Blood pixels'], Hb))
print(np.corrcoef(df['Division Mean of Blood pixels by BG'], Hb))
print(np.corrcoef(df['Division Median of Blood pixels by BG'], Hb))
print(np.corrcoef(df['Division Mode of Blood pixels by BG'], Hb))
print(stat.spearmanr(df['Mean of Blood pixels'], Hb))
print(stat.spearmanr(df['Median of Blood pixels'], Hb))
print(stat.spearmanr(df['Mode of Blood pixels'], Hb))
print(stat.spearmanr(df['Division Mean of Blood pixels by BG'], Hb))
print(stat.spearmanr(df['Division Median of Blood pixels by BG'], Hb))
print(stat.spearmanr(df['Division Mode of Blood pixels by BG'], Hb))

print(np.corrcoef(df['Mean of Blood pixels_2'], Hb))
print(np.corrcoef(df['Median of Blood pixels_2'], Hb))
print(np.corrcoef(df['Mode of Blood pixels_2'], Hb))
print(np.corrcoef(df['Division Mean of Blood pixels by BG_2'], Hb))
print(np.corrcoef(df['Division Median of Blood pixels by BG_2'], Hb))
print(np.corrcoef(df['Division Mode of Blood pixels by BG_2'], Hb))
print(stat.spearmanr(df['Mean of Blood pixels_2'], Hb))
print(stat.spearmanr(df['Median of Blood pixels_2'], Hb))
print(stat.spearmanr(df['Mode of Blood pixels_2'], Hb))
print(stat.spearmanr(df['Division Mean of Blood pixels by BG_2'], Hb))
print(stat.spearmanr(df['Division Median of Blood pixels by BG_2'], Hb))
print(stat.spearmanr(df['Division Mode of Blood pixels by BG_2'], Hb))
reg = LinearRegression().fit(df, Hb)
print(reg.coef_)
print(reg.intercept_)
# reg.predict
# print(df_660.head())
print(np.corrcoef(df_1['Mean of Blood pixels'], Hb))
print(np.corrcoef(df_1['Median of Blood pixels'], Hb))
print(np.corrcoef(df_1['Mode of Blood pixels'], Hb))
print(np.corrcoef(df_1['Division Mean of Blood pixels by BG'], Hb))
print(np.corrcoef(df_1['Division Median of Blood pixels by BG'], Hb))
print(np.corrcoef(df_1['Division Mode of Blood pixels by BG'], Hb))
print(stat.spearmanr(df_1['Mean of Blood pixels'], Hb))
print(stat.spearmanr(df_1['Median of Blood pixels'], Hb))
print(stat.spearmanr(df_1['Mode of Blood pixels'], Hb))
print(stat.spearmanr(df_1['Division Mean of Blood pixels by BG'], Hb))
print(stat.spearmanr(df_1['Division Median of Blood pixels by BG'], Hb))
print(stat.spearmanr(df_1['Division Mode of Blood pixels by BG'], Hb))

print(np.corrcoef(df_2['Mean of Blood pixels'], Hb))
print(np.corrcoef(df_2['Median of Blood pixels'], Hb))
print(np.corrcoef(df_2['Mode of Blood pixels'], Hb))
print(np.corrcoef(df_2['Division Mean of Blood pixels by BG'], Hb))
print(np.corrcoef(df_2['Division Median of Blood pixels by BG'], Hb))
print(np.corrcoef(df_2['Division Mode of Blood pixels by BG'], Hb))
print(stat.spearmanr(df_2['Mean of Blood pixels'], Hb))
print(stat.spearmanr(df_2['Median of Blood pixels'], Hb))
print(stat.spearmanr(df_2['Mode of Blood pixels'], Hb))
print(stat.spearmanr(df_2['Division Mean of Blood pixels by BG'], Hb))
print(stat.spearmanr(df_2['Division Median of Blood pixels by BG'], Hb))
print(stat.spearmanr(df_2['Division Mode of Blood pixels by BG'], Hb))

In [ ]:
### 800 nm
imdr = '/Volumes/KURAPIKA4/Master_Thesis/Iron dataset/Monochrome/110219/800/Noise/'
files_1 = []
[files_1.extend(glob.glob(imdr + '*.tif'))]
images_1 = [cv.imread(file, 0) for file in files_1]
noise_1 = np.zeros((2048, 2448))
for img in images_1:
    noise_1 += img
noise_1 = noise_1/len(images_1)
# Blank images
imdr = '/Volumes/KURAPIKA4/Master_Thesis/Iron dataset/Monochrome/110219/800/Blank/'
files_1 = []
[files_1.extend(glob.glob(imdr + '*.tif'))]
images_1 = [cv.imread(file, 0) for file in files_1]
blank_1 = np.zeros((2048, 2448))
for img in images_1:
    blank_1 += img
blank_1 = blank_1/len(images_1)
# Read all images
imdr = '/Volumes/KURAPIKA4/Master_Thesis/Iron dataset/Monochrome/110219/800/Samples/'
files = []
[files.extend(glob.glob(imdr + '*.tif'))]
images = [cv.imread(file, 0) for file in files]
# print(len(images))
blood_mean = []
blood_median = []
blood_mode = []
bld_mean_bld_mean = []
bld_median_bld_median = []
bld_mode_bld_mode = []
for img in images:
    # illumination correction
#     img_ill = (img-noise_1)/(blank_1-noise_1)
    img_ill = img

    # apply threshold to get the binary image
    th = threshold_otsu(img)
    thg = img > th
    th1 = np.invert(thg)
    
    img_I = img*th1
    img_BG = img_ill*thg
    # apply threshold
    thresh = threshold_otsu(img_I)
    bw = closing(img_I > thresh, square(1))
    bw = opening(bw, square(2))
    mask = np.ones(bw.shape)
    mask[1400:2047, 0:200] = 0
    # print(mask)
    mask = mask.astype(bool)
    # remove artifacts connected to image border
    cleared = clear_border(bw, mask = mask)
    # label image regions
    label_image = label(cleared)
    for region in regionprops(label_image):
        # take regions with large enough areas
        if region.area >= 1000000:
            # find the bounding box
            minr, minc, maxr, maxc = region.bbox
    # crop the images to the found bounding box
    img_fin = (cleared*img_ill*th1)[minr:maxr, minc:maxc]
    img_fig = img_BG[minr:maxr, minc:maxc]

    plt.subplot(1,2,1), plt.imshow(img_fin, cmap = 'gray')
    plt.subplot(1,2,2), plt.imshow(img_fig, cmap = 'gray')
    plt.show()

    # Find the mean, median and mode of the blood spot
    blood_mean.append(np.mean(img_fin[img_fin>0]))
    blood_median.append(np.median(img_fin[img_fin>0]))
    blood_mode.append(statistics.mode(img_fin[img_fin>0]))
    bld_mean_bld_mean.append(np.mean(img_fin[img_fin>0])/np.mean(img_fig[img_fig>0]))
    bld_median_bld_median.append(np.median(img_fin[img_fin>0])/np.median(img_fig[img_fig>0]))
    bld_mode_bld_mode.append(statistics.mode(img_fin[img_fin>0])/statistics.mode(img_fig[img_fig>0]))

# print(blood_mean)
# print(blood_median)
# print(blood_mode)

# print(bld_mean_bld_mean)
# print(bld_median_bld_median)
# print(bld_mode_bld_mode)
df_800 = pd.DataFrame(np.column_stack([blood_mean[::2], blood_median[::2], blood_mode[::2], bld_mean_bld_mean[::2], bld_median_bld_median[::2], bld_mode_bld_mode[::2], blood_mean[1::2], blood_median[1::2], blood_mode[1::2], bld_mean_bld_mean[1::2], bld_median_bld_median[1::2], bld_mode_bld_mode[1::2]]), columns = ['Mean of Blood pixels', 'Median of Blood pixels', 'Mode of Blood pixels', 'Division Mean of Blood pixels by BG', 'Division Median of Blood pixels by BG', 'Division Mode of Blood pixels by BG', 'Mean of Blood pixels_2', 'Median of Blood pixels_2', 'Mode of Blood pixels_2', 'Division Mean of Blood pixels by BG_2', 'Division Median of Blood pixels by BG_2', 'Division Mode of Blood pixels by BG_2'])
# print(df_660)

In [ ]:
### 800 nm by Set
# Read Noise images
imdr = '/Volumes/KURAPIKA4/Master_Thesis/Iron dataset/Monochrome/110219/800/Noise/'
files_1 = []
[files_1.extend(glob.glob(imdr + '*1.tif'))]
images_1 = [cv.imread(file, 0) for file in files_1]
noise_1 = np.zeros((2048, 2448))
for img in images_1:
    noise_1 += img
noise_1 = noise_1/len(images_1)
files_2 = []
[files_2.extend(glob.glob(imdr + '*2.tif'))]
images_2 = [cv.imread(file, 0) for file in files_2]
noise_2 = np.zeros((2048, 2448))
for img in images_2:
    noise_2 += img
noise_2 = noise_2/len(images_2)
# Read Blank images
imdr = '/Volumes/KURAPIKA4/Master_Thesis/Iron dataset/Monochrome/110219/800/Blank/'
files_1 = []
[files_1.extend(glob.glob(imdr + '*1.tif'))]
images_1 = [cv.imread(file, 0) for file in files_1]
blank_1 = np.zeros((2048, 2448))
for img in images_1:
    blank_1 += img
blank_1 = blank_1/len(images_1)
files_2 = []
[files_2.extend(glob.glob(imdr + '*2.tif'))]
images_2 = [cv.imread(file, 0) for file in files_2]
blank_2 = np.zeros((2048, 2448))
for img in images_2:
    blank_2 += img
blank_2 = blank_2/len(images_2)
# print(np.bincount(np.ravel(noise_1 == noise_2)))
# print(np.bincount(np.ravel(blank_1 == blank_2)))
### Set 1
# Read all images
imdr = '/Volumes/KURAPIKA4/Master_Thesis/Iron dataset/Monochrome/110219/800/Samples/'
files = []
[files.extend(glob.glob(imdr + '*1.tif'))]
images = [cv.imread(file, 0) for file in files]
blood_mean = []
blood_median = []
blood_mode = []
bld_mean_bld_mean = []
bld_median_bld_median = []
bld_mode_bld_mode = []
for img in images:
    # illumination correction
#     img_ill = (img-noise_1)/(blank_1-noise_1)
    img_ill = img
    
    # apply threshold to get the binary image
    th = threshold_otsu(img)
    thg = img > th
    th1 = np.invert(thg)
    
    img_I = img*th1
    img_BG = img_ill*thg
    # apply threshold
    thresh = threshold_otsu(img_I)
    bw = closing(img_I > thresh, square(1))
    bw = opening(bw, square(2))
    mask = np.ones(bw.shape)
    mask[1400:2047, 0:200] = 0
    # print(mask)
    mask = mask.astype(bool)
    # remove artifacts connected to image border
    cleared = clear_border(bw, mask = mask)
    # label image regions
    label_image = label(cleared)
    for region in regionprops(label_image):
        # take regions with large enough areas
        if region.area >= 1000000:
            # find the bounding box
            minr, minc, maxr, maxc = region.bbox
    # crop the images to the found bounding box
    img_fin = (cleared*img_ill*th1)[minr:maxr, minc:maxc]
    img_fig = img_BG[minr:maxr, minc:maxc]
    
    plt.subplot(1,2,1), plt.imshow(img_fin, cmap = 'gray')
    plt.subplot(1,2,2), plt.imshow(img_fig, cmap = 'gray')
    plt.show()

    # Find the mean, median and mode of the blood spot
    blood_mean.append(np.mean(img_fin[img_fin>0]))
    blood_median.append(np.median(img_fin[img_fin>0]))
    blood_mode.append(statistics.mode(img_fin[img_fin>0]))
    bld_mean_bld_mean.append(np.mean(img_fin[img_fin>0])/np.mean(img_fig[img_fig>0]))
    bld_median_bld_median.append(np.median(img_fin[img_fin>0])/np.median(img_fig[img_fig>0]))
    bld_mode_bld_mode.append(statistics.mode(img_fin[img_fin>0])/statistics.mode(img_fig[img_fig>0]))
df_800_1 = pd.DataFrame(np.column_stack([blood_mean, blood_median, blood_mode, bld_mean_bld_mean, bld_median_bld_median, bld_mode_bld_mode]), columns = ['Mean of Blood pixels', 'Median of Blood pixels', 'Mode of Blood pixels', 'Division Mean of Blood pixels by BG', 'Division Median of Blood pixels by BG', 'Division Mode of Blood pixels by BG'])
## Set 2
files = []
[files.extend(glob.glob(imdr + '*2.tif'))]
images = [cv.imread(file, 0) for file in files]
blood_mean = []
blood_median = []
blood_mode = []
bld_mean_bld_mean = []
bld_median_bld_median = []
bld_mode_bld_mode = []
for img in images:
    # illumination correction
    img_ill = (img-noise_2)/(blank_2-noise_2)
    
    # apply threshold to get the binary image
    th = threshold_otsu(img)
    thg = img > th
    th1 = np.invert(thg)
    
    img_I = img*th1
    img_BG = img_ill*thg
    # apply threshold
    thresh = threshold_otsu(img_I)
    bw = closing(img_I > thresh, square(1))
    bw = opening(bw, square(2))
    mask = np.ones(bw.shape)
    mask[1400:2047, 0:200] = 0
    # print(mask)
    mask = mask.astype(bool)
    # remove artifacts connected to image border
    cleared = clear_border(bw, mask = mask)
    # label image regions
    label_image = label(cleared)
    for region in regionprops(label_image):
        # take regions with large enough areas
        if region.area >= 1000000:
            # find the bounding box
            minr, minc, maxr, maxc = region.bbox
    # crop the images to the found bounding box
    img_fin = (cleared*img_ill*th1)[minr:maxr, minc:maxc]
    img_fig = img_BG[minr:maxr, minc:maxc]
    
    plt.subplot(1,2,1), plt.imshow(img_fin, cmap = 'gray')
    plt.subplot(1,2,2), plt.imshow(img_fig, cmap = 'gray')
    plt.show()

    # Find the mean, median and mode of the blood spot
    blood_mean.append(np.mean(img_fin[img_fin>0]))
    blood_median.append(np.median(img_fin[img_fin>0]))
    blood_mode.append(statistics.mode(img_fin[img_fin>0]))
    bld_mean_bld_mean.append(np.mean(img_fin[img_fin>0])/np.mean(img_fig[img_fig>0]))
    bld_median_bld_median.append(np.median(img_fin[img_fin>0])/np.median(img_fig[img_fig>0]))
    bld_mode_bld_mode.append(statistics.mode(img_fin[img_fin>0])/statistics.mode(img_fig[img_fig>0]))
df_800_2 = pd.DataFrame(np.column_stack([blood_mean, blood_median, blood_mode, bld_mean_bld_mean, bld_median_bld_median, bld_mode_bld_mode]), columns = ['Mean of Blood pixels', 'Median of Blood pixels', 'Mode of Blood pixels', 'Division Mean of Blood pixels by BG', 'Division Median of Blood pixels by BG', 'Division Mode of Blood pixels by BG'])

In [ ]:
with open('DF_800.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([df_800, df_800_1, df_800_2], f)

In [ ]:
### Linear Regression
with open('DF_800.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    df_800, df_800_1, df_800_2 = pickle.load(f)
    
df = df_800
df_1 = df_800_1
df_2 = df_800_2
print(np.corrcoef(df['Mean of Blood pixels'], Hb))
print(np.corrcoef(df['Median of Blood pixels'], Hb))
print(np.corrcoef(df['Mode of Blood pixels'], Hb))
print(np.corrcoef(df['Division Mean of Blood pixels by BG'], Hb))
print(np.corrcoef(df['Division Median of Blood pixels by BG'], Hb))
print(np.corrcoef(df['Division Mode of Blood pixels by BG'], Hb))
print(stat.spearmanr(df['Mean of Blood pixels'], Hb))
print(stat.spearmanr(df['Median of Blood pixels'], Hb))
print(stat.spearmanr(df['Mode of Blood pixels'], Hb))
print(stat.spearmanr(df['Division Mean of Blood pixels by BG'], Hb))
print(stat.spearmanr(df['Division Median of Blood pixels by BG'], Hb))
print(stat.spearmanr(df['Division Mode of Blood pixels by BG'], Hb))

print(np.corrcoef(df['Mean of Blood pixels_2'], Hb))
print(np.corrcoef(df['Median of Blood pixels_2'], Hb))
print(np.corrcoef(df['Mode of Blood pixels_2'], Hb))
print(np.corrcoef(df['Division Mean of Blood pixels by BG_2'], Hb))
print(np.corrcoef(df['Division Median of Blood pixels by BG_2'], Hb))
print(np.corrcoef(df['Division Mode of Blood pixels by BG_2'], Hb))
print(stat.spearmanr(df['Mean of Blood pixels_2'], Hb))
print(stat.spearmanr(df['Median of Blood pixels_2'], Hb))
print(stat.spearmanr(df['Mode of Blood pixels_2'], Hb))
print(stat.spearmanr(df['Division Mean of Blood pixels by BG_2'], Hb))
print(stat.spearmanr(df['Division Median of Blood pixels by BG_2'], Hb))
print(stat.spearmanr(df['Division Mode of Blood pixels by BG_2'], Hb))
reg = LinearRegression().fit(df, Hb)
print(reg.coef_)
print(reg.intercept_)
# reg.predict
# print(df_660.head())
print(np.corrcoef(df_1['Mean of Blood pixels'], Hb))
print(np.corrcoef(df_1['Median of Blood pixels'], Hb))
print(np.corrcoef(df_1['Mode of Blood pixels'], Hb))
print(np.corrcoef(df_1['Division Mean of Blood pixels by BG'], Hb))
print(np.corrcoef(df_1['Division Median of Blood pixels by BG'], Hb))
print(np.corrcoef(df_1['Division Mode of Blood pixels by BG'], Hb))
print(stat.spearmanr(df_1['Mean of Blood pixels'], Hb))
print(stat.spearmanr(df_1['Median of Blood pixels'], Hb))
print(stat.spearmanr(df_1['Mode of Blood pixels'], Hb))
print(stat.spearmanr(df_1['Division Mean of Blood pixels by BG'], Hb))
print(stat.spearmanr(df_1['Division Median of Blood pixels by BG'], Hb))
print(stat.spearmanr(df_1['Division Mode of Blood pixels by BG'], Hb))

print(np.corrcoef(df_2['Mean of Blood pixels'], Hb))
print(np.corrcoef(df_2['Median of Blood pixels'], Hb))
print(np.corrcoef(df_2['Mode of Blood pixels'], Hb))
print(np.corrcoef(df_2['Division Mean of Blood pixels by BG'], Hb))
print(np.corrcoef(df_2['Division Median of Blood pixels by BG'], Hb))
print(np.corrcoef(df_2['Division Mode of Blood pixels by BG'], Hb))
print(stat.spearmanr(df_2['Mean of Blood pixels'], Hb))
print(stat.spearmanr(df_2['Median of Blood pixels'], Hb))
print(stat.spearmanr(df_2['Mode of Blood pixels'], Hb))
print(stat.spearmanr(df_2['Division Mean of Blood pixels by BG'], Hb))
print(stat.spearmanr(df_2['Division Median of Blood pixels by BG'], Hb))
print(stat.spearmanr(df_2['Division Mode of Blood pixels by BG'], Hb))

In [ ]:
### 940 nm
imdr = '/Volumes/KURAPIKA4/Master_Thesis/Iron dataset/Monochrome/110219/940/Noise/'
files_1 = []
[files_1.extend(glob.glob(imdr + '*.tif'))]
images_1 = [cv.imread(file, 0) for file in files_1]
noise_1 = np.zeros((2048, 2448))
for img in images_1:
    noise_1 += img
noise_1 = noise_1/len(images_1)
# Blank images
imdr = '/Volumes/KURAPIKA4/Master_Thesis/Iron dataset/Monochrome/110219/940/Blank/'
files_1 = []
[files_1.extend(glob.glob(imdr + '*.tif'))]
images_1 = [cv.imread(file, 0) for file in files_1]
blank_1 = np.zeros((2048, 2448))
for img in images_1:
    blank_1 += img
blank_1 = blank_1/len(images_1)
# Read all images
imdr = '/Volumes/KURAPIKA4/Master_Thesis/Iron dataset/Monochrome/110219/940/Samples/'
files = []
[files.extend(glob.glob(imdr + '*.tif'))]
images = [cv.imread(file, 0) for file in files]
# print(len(images))
blood_mean = []
blood_median = []
blood_mode = []
bld_mean_bld_mean = []
bld_median_bld_median = []
bld_mode_bld_mode = []
for img in images:
    # illumination correction
    img_ill = (img-noise_1)/(blank_1-noise_1)

    # apply threshold to get the binary image
    th = threshold_otsu(img)
    thg = img > th
    th1 = np.invert(thg)
    
    img_I = img*th1
    img_BG = img_ill*thg
    # apply threshold
    thresh = threshold_otsu(img_I)
    bw = closing(img_I > thresh, square(1))
    bw = opening(bw, square(2))
    mask = np.ones(bw.shape)
    mask[1400:2047, 0:200] = 0
    # print(mask)
    mask = mask.astype(bool)
    # remove artifacts connected to image border
    cleared = clear_border(bw, mask = mask)
    # label image regions
    label_image = label(cleared)
    for region in regionprops(label_image):
        # take regions with large enough areas
        if region.area >= 1000000:
            # find the bounding box
            minr, minc, maxr, maxc = region.bbox
    # crop the images to the found bounding box
    img_fin = (cleared*img_ill*th1)[minr:maxr, minc:maxc]
    img_fig = img_BG[minr:maxr, minc:maxc]

    plt.subplot(1,2,1), plt.imshow(img_fin, cmap = 'gray')
    plt.subplot(1,2,2), plt.imshow(img_fig, cmap = 'gray')
    plt.show()

    # Find the mean, median and mode of the blood spot
    blood_mean.append(np.mean(img_fin[img_fin>0]))
    blood_median.append(np.median(img_fin[img_fin>0]))
    blood_mode.append(statistics.mode(img_fin[img_fin>0]))
    bld_mean_bld_mean.append(np.mean(img_fin[img_fin>0])/np.mean(img_fig[img_fig>0]))
    bld_median_bld_median.append(np.median(img_fin[img_fin>0])/np.median(img_fig[img_fig>0]))
    bld_mode_bld_mode.append(statistics.mode(img_fin[img_fin>0])/statistics.mode(img_fig[img_fig>0]))

# print(blood_mean)
# print(blood_median)
# print(blood_mode)
# print(bld_mean_bld_mean)
# print(bld_median_bld_median)
# print(bld_mode_bld_mode)
df_940 = pd.DataFrame(np.column_stack([blood_mean[::2], blood_median[::2], blood_mode[::2], bld_mean_bld_mean[::2], bld_median_bld_median[::2], bld_mode_bld_mode[::2], blood_mean[1::2], blood_median[1::2], blood_mode[1::2], bld_mean_bld_mean[1::2], bld_median_bld_median[1::2], bld_mode_bld_mode[1::2]]), columns = ['Mean of Blood pixels', 'Median of Blood pixels', 'Mode of Blood pixels', 'Division Mean of Blood pixels by BG', 'Division Median of Blood pixels by BG', 'Division Mode of Blood pixels by BG', 'Mean of Blood pixels_2', 'Median of Blood pixels_2', 'Mode of Blood pixels_2', 'Division Mean of Blood pixels by BG_2', 'Division Median of Blood pixels by BG_2', 'Division Mode of Blood pixels by BG_2'])
# print(df_660)

In [ ]:
### 940 nm by Set
# Read Noise images
imdr = '/Volumes/KURAPIKA4/Master_Thesis/Iron dataset/Monochrome/110219/940/Noise/'
files_1 = []
[files_1.extend(glob.glob(imdr + '*1.tif'))]
images_1 = [cv.imread(file, 0) for file in files_1]
noise_1 = np.zeros((2048, 2448))
for img in images_1:
    noise_1 += img
noise_1 = noise_1/len(images_1)
files_2 = []
[files_2.extend(glob.glob(imdr + '*2.tif'))]
images_2 = [cv.imread(file, 0) for file in files_2]
noise_2 = np.zeros((2048, 2448))
for img in images_2:
    noise_2 += img
noise_2 = noise_2/len(images_2)
# Read Blank images
imdr = '/Volumes/KURAPIKA4/Master_Thesis/Iron dataset/Monochrome/110219/940/Blank/'
files_1 = []
[files_1.extend(glob.glob(imdr + '*1.tif'))]
images_1 = [cv.imread(file, 0) for file in files_1]
blank_1 = np.zeros((2048, 2448))
for img in images_1:
    blank_1 += img
blank_1 = blank_1/len(images_1)
files_2 = []
[files_2.extend(glob.glob(imdr + '*2.tif'))]
images_2 = [cv.imread(file, 0) for file in files_2]
blank_2 = np.zeros((2048, 2448))
for img in images_2:
    blank_2 += img
blank_2 = blank_2/len(images_2)
# print(np.bincount(np.ravel(noise_1 == noise_2)))
# print(np.bincount(np.ravel(blank_1 == blank_2)))
### Set 1
# Read all images
imdr = '/Volumes/KURAPIKA4/Master_Thesis/Iron dataset/Monochrome/110219/940/Samples/'
files = []
[files.extend(glob.glob(imdr + '*1.tif'))]
images = [cv.imread(file, 0) for file in files]
blood_mean = []
blood_median = []
blood_mode = []
bld_mean_bld_mean = []
bld_median_bld_median = []
bld_mode_bld_mode = []
for img in images:
    # illumination correction
    img_ill = (img-noise_1)/(blank_1-noise_1)
    
    # apply threshold to get the binary image
    th = threshold_otsu(img)
    thg = img > th
    th1 = np.invert(thg)
    
    img_I = img*th1
    img_BG = img_ill*thg
    # apply threshold
    thresh = threshold_otsu(img_I)
    bw = closing(img_I > thresh, square(1))
    bw = opening(bw, square(2))
    mask = np.ones(bw.shape)
    mask[1400:2047, 0:200] = 0
    # print(mask)
    mask = mask.astype(bool)
    # remove artifacts connected to image border
    cleared = clear_border(bw, mask = mask)
    # label image regions
    label_image = label(cleared)
    for region in regionprops(label_image):
        # take regions with large enough areas
        if region.area >= 1000000:
            # find the bounding box
            minr, minc, maxr, maxc = region.bbox
    # crop the images to the found bounding box
    img_fin = (cleared*img_ill*th1)[minr:maxr, minc:maxc]
    img_fig = img_BG[minr:maxr, minc:maxc]
    
    plt.subplot(1,2,1), plt.imshow(img_fin, cmap = 'gray')
    plt.subplot(1,2,2), plt.imshow(img_fig, cmap = 'gray')
    plt.show()

    # Find the mean, median and mode of the blood spot
    blood_mean.append(np.mean(img_fin[img_fin>0]))
    blood_median.append(np.median(img_fin[img_fin>0]))
    blood_mode.append(statistics.mode(img_fin[img_fin>0]))
    bld_mean_bld_mean.append(np.mean(img_fin[img_fin>0])/np.mean(img_fig[img_fig>0]))
    bld_median_bld_median.append(np.median(img_fin[img_fin>0])/np.median(img_fig[img_fig>0]))
    bld_mode_bld_mode.append(statistics.mode(img_fin[img_fin>0])/statistics.mode(img_fig[img_fig>0]))
df_940_1 = pd.DataFrame(np.column_stack([blood_mean, blood_median, blood_mode, bld_mean_bld_mean, bld_median_bld_median, bld_mode_bld_mode]), columns = ['Mean of Blood pixels', 'Median of Blood pixels', 'Mode of Blood pixels', 'Division Mean of Blood pixels by BG', 'Division Median of Blood pixels by BG', 'Division Mode of Blood pixels by BG'])
## Set 2
files = []
[files.extend(glob.glob(imdr + '*2.tif'))]
images = [cv.imread(file, 0) for file in files]
blood_mean = []
blood_median = []
blood_mode = []
bld_mean_bld_mean = []
bld_median_bld_median = []
bld_mode_bld_mode = []
for img in images:
    # illumination correction
    img_ill = (img-noise_2)/(blank_2-noise_2)
    
    # apply threshold to get the binary image
    th = threshold_otsu(img)
    thg = img > th
    th1 = np.invert(thg)
    
    img_I = img*th1
    img_BG = img_ill*thg
    # apply threshold
    thresh = threshold_otsu(img_I)
    bw = closing(img_I > thresh, square(1))
    bw = opening(bw, square(2))
    mask = np.ones(bw.shape)
    mask[1400:2047, 0:200] = 0
    # print(mask)
    mask = mask.astype(bool)
    # remove artifacts connected to image border
    cleared = clear_border(bw, mask = mask)
    # label image regions
    label_image = label(cleared)
    for region in regionprops(label_image):
        # take regions with large enough areas
        if region.area >= 1000000:
            # find the bounding box
            minr, minc, maxr, maxc = region.bbox
    # crop the images to the found bounding box
    img_fin = (cleared*img_ill*th1)[minr:maxr, minc:maxc]
    img_fig = img_BG[minr:maxr, minc:maxc]
    
    plt.subplot(1,2,1), plt.imshow(img_fin, cmap = 'gray')
    plt.subplot(1,2,2), plt.imshow(img_fig, cmap = 'gray')
    plt.show()

    # Find the mean, median and mode of the blood spot
    blood_mean.append(np.mean(img_fin[img_fin>0]))
    blood_median.append(np.median(img_fin[img_fin>0]))
    blood_mode.append(statistics.mode(img_fin[img_fin>0]))
    bld_mean_bld_mean.append(np.mean(img_fin[img_fin>0])/np.mean(img_fig[img_fig>0]))
    bld_median_bld_median.append(np.median(img_fin[img_fin>0])/np.median(img_fig[img_fig>0]))
    bld_mode_bld_mode.append(statistics.mode(img_fin[img_fin>0])/statistics.mode(img_fig[img_fig>0]))
df_940_2 = pd.DataFrame(np.column_stack([blood_mean, blood_median, blood_mode, bld_mean_bld_mean, bld_median_bld_median, bld_mode_bld_mode]), columns = ['Mean of Blood pixels', 'Median of Blood pixels', 'Mode of Blood pixels', 'Division Mean of Blood pixels by BG', 'Division Median of Blood pixels by BG', 'Division Mode of Blood pixels by BG'])

In [ ]:
with open('DF_940.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([df_940, df_940_1, df_940_2], f)

In [ ]:
### Linear Regression
with open('DF_940.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    df_940, df_940_1, df_940_2 = pickle.load(f)
    
df = df_940
df_1 = df_940_1
df_2 = df_940_2
print(np.corrcoef(df['Mean of Blood pixels'], Hb))
print(np.corrcoef(df['Median of Blood pixels'], Hb))
print(np.corrcoef(df['Mode of Blood pixels'], Hb))
print(np.corrcoef(df['Division Mean of Blood pixels by BG'], Hb))
print(np.corrcoef(df['Division Median of Blood pixels by BG'], Hb))
print(np.corrcoef(df['Division Mode of Blood pixels by BG'], Hb))
print(stat.spearmanr(df['Mean of Blood pixels'], Hb))
print(stat.spearmanr(df['Median of Blood pixels'], Hb))
print(stat.spearmanr(df['Mode of Blood pixels'], Hb))
print(stat.spearmanr(df['Division Mean of Blood pixels by BG'], Hb))
print(stat.spearmanr(df['Division Median of Blood pixels by BG'], Hb))
print(stat.spearmanr(df['Division Mode of Blood pixels by BG'], Hb))

print(np.corrcoef(df['Mean of Blood pixels_2'], Hb))
print(np.corrcoef(df['Median of Blood pixels_2'], Hb))
print(np.corrcoef(df['Mode of Blood pixels_2'], Hb))
print(np.corrcoef(df['Division Mean of Blood pixels by BG_2'], Hb))
print(np.corrcoef(df['Division Median of Blood pixels by BG_2'], Hb))
print(np.corrcoef(df['Division Mode of Blood pixels by BG_2'], Hb))
print(stat.spearmanr(df['Mean of Blood pixels_2'], Hb))
print(stat.spearmanr(df['Median of Blood pixels_2'], Hb))
print(stat.spearmanr(df['Mode of Blood pixels_2'], Hb))
print(stat.spearmanr(df['Division Mean of Blood pixels by BG_2'], Hb))
print(stat.spearmanr(df['Division Median of Blood pixels by BG_2'], Hb))
print(stat.spearmanr(df['Division Mode of Blood pixels by BG_2'], Hb))
reg = LinearRegression().fit(df, Hb)
print(reg.coef_)
print(reg.intercept_)
# reg.predict
# print(df_660.head())
print(np.corrcoef(df_1['Mean of Blood pixels'], Hb))
print(np.corrcoef(df_1['Median of Blood pixels'], Hb))
print(np.corrcoef(df_1['Mode of Blood pixels'], Hb))
print(np.corrcoef(df_1['Division Mean of Blood pixels by BG'], Hb))
print(np.corrcoef(df_1['Division Median of Blood pixels by BG'], Hb))
print(np.corrcoef(df_1['Division Mode of Blood pixels by BG'], Hb))
print(stat.spearmanr(df_1['Mean of Blood pixels'], Hb))
print(stat.spearmanr(df_1['Median of Blood pixels'], Hb))
print(stat.spearmanr(df_1['Mode of Blood pixels'], Hb))
print(stat.spearmanr(df_1['Division Mean of Blood pixels by BG'], Hb))
print(stat.spearmanr(df_1['Division Median of Blood pixels by BG'], Hb))
print(stat.spearmanr(df_1['Division Mode of Blood pixels by BG'], Hb))

print(np.corrcoef(df_2['Mean of Blood pixels'], Hb))
print(np.corrcoef(df_2['Median of Blood pixels'], Hb))
print(np.corrcoef(df_2['Mode of Blood pixels'], Hb))
print(np.corrcoef(df_2['Division Mean of Blood pixels by BG'], Hb))
print(np.corrcoef(df_2['Division Median of Blood pixels by BG'], Hb))
print(np.corrcoef(df_2['Division Mode of Blood pixels by BG'], Hb))
print(stat.spearmanr(df_2['Mean of Blood pixels'], Hb))
print(stat.spearmanr(df_2['Median of Blood pixels'], Hb))
print(stat.spearmanr(df_2['Mode of Blood pixels'], Hb))
print(stat.spearmanr(df_2['Division Mean of Blood pixels by BG'], Hb))
print(stat.spearmanr(df_2['Division Median of Blood pixels by BG'], Hb))
print(stat.spearmanr(df_2['Division Mode of Blood pixels by BG'], Hb))